# Flow: Dual-Pile State Machine

**Architecture**: Flow combines two specialized Pile instances:
- `progressions`: Named workflow stages (ordered)
- `items`: Shared item storage

**Key Features**:
- M:N relationships (items in multiple progressions)
- Named state access: `flow.get_progression("pending")` → O(1)
- Independent lifecycles (items persist across stage transitions)
- UUID-based references (serialization-friendly)

**Pattern**: Workflow state machine where progressions define stages, items are work units.

In [ ]:
# Imports
from lionherd_core.base import Element, Flow, Progression

# Test item types
class Task(Element):
    description: str = "task"
    priority: int = 0

## 1. Construction

Create Flow with optional initial items and type validation.

In [ ]:
# Empty flow
flow1 = Flow[Task, Progression]()
print(f"Empty: {flow1}")

# Flow with initial items
tasks = [Task(description=f"task{i}", priority=i) for i in range(3)]
flow2 = Flow[Task, Progression](items=tasks, name="deployment")
print(f"With items: {flow2}")
print(f"Items in pile: {len(flow2.items)}")

## 2. Add Named Progressions

Progressions define workflow stages with unique names.

In [ ]:
# Define workflow stages
flow = Flow[Task, Progression](name="workflow")

flow.add_progression(Progression(name="pending"))
flow.add_progression(Progression(name="active"))
flow.add_progression(Progression(name="completed"))

print(f"Progressions: {len(flow.progressions)}")
print(f"Names registered: {list(flow._progression_names.keys())}")

## 3. Add Items to Pile and Progressions

Items go into shared pile, optionally assigned to progressions.

In [ ]:
# Add to pile only
task1 = Task(description="task1", priority=1)
flow.add_item(task1)
print(f"Item in pile: {task1.id in flow.items}")
print(f"In pending: {task1.id in flow.get_progression('pending')}")

# Add to pile and progression (state assignment)
task2 = Task(description="task2", priority=2)
flow.add_item(task2, progression_ids="pending")
print(f"\nTask2 in pile: {task2.id in flow.items}")
print(f"Task2 in pending: {task2.id in flow.get_progression('pending')}")
print(f"Pending count: {len(flow.get_progression('pending'))}")

## 4. M:N Relationships

Items can exist in multiple progressions simultaneously.

In [ ]:
# Add item to multiple progressions
task3 = Task(description="task3", priority=3)
flow.add_item(task3, progression_ids=["pending", "active"])

print(f"Task3 in pending: {task3.id in flow.get_progression('pending')}")
print(f"Task3 in active: {task3.id in flow.get_progression('active')}")
print(f"Task3 in completed: {task3.id in flow.get_progression('completed')}")

# Single item, stored once
print(f"\nItems in pile: {len(flow.items)}")
print(f"Total progression memberships: {sum(len(p) for p in flow.progressions)}")

## 5. Named Access and Queries

Access progressions by name for ergonomic workflow queries.

In [ ]:
# Get progression by name (O(1))
pending = flow.get_progression("pending")
print(f"Pending progression: {pending.name}, items: {len(pending)}")

# Retrieve items from progression
pending_tasks = [flow.items[item_id] for item_id in pending.order]
print(f"\nPending tasks:")
for task in pending_tasks:
    print(f"  - {task.description} (priority: {task.priority})")

# Query all stages
print(f"\nWorkflow state:")
for prog in flow.progressions:
    print(f"  {prog.name}: {len(prog)} items")

## 6. Workflow State Transitions

Move items between progressions (state machine pattern).

In [ ]:
# State transition: pending → active
pending_prog = flow.get_progression("pending")
active_prog = flow.get_progression("active")

# Get first pending task
if len(pending_prog) > 0:
    task_id = pending_prog.order[0]
    task = flow.items[task_id]
    
    print(f"Transitioning: {task.description}")
    print(f"Before - Pending: {len(pending_prog)}, Active: {len(active_prog)}")
    
    # Move between stages
    pending_prog.remove(task_id)
    active_prog.append(task_id)
    
    print(f"After - Pending: {len(pending_prog)}, Active: {len(active_prog)}")
    
    # Item persists in pile (single source of truth)
    print(f"Item still in pile: {task_id in flow.items}")

## 7. Serialization

Flow serializes to dict with proper Pile serialization.

In [ ]:
# Serialize
data = flow.to_dict()
print(f"Serialized keys: {list(data.keys())}")
print(f"Items type: {type(data['items'])}")
print(f"Progressions type: {type(data['progressions'])}")

# Deserialize
flow_restored = Flow.from_dict(data)
print(f"\nRestored: {flow_restored}")
print(f"Name index rebuilt: {list(flow_restored._progression_names.keys())}")

# Verify named access works after deserialization
restored_pending = flow_restored.get_progression("pending")
print(f"Access by name works: {restored_pending.name}")

## 8. Complete Workflow Example

End-to-end deployment pipeline demonstration.

In [ ]:
# Create deployment pipeline
pipeline = Flow[Task, Progression](name="deploy_pipeline")

# Define stages
for stage in ["testing", "staging", "production"]:
    pipeline.add_progression(Progression(name=stage))

# Add deployment tasks
deployments = [
    Task(description="api_service", priority=1),
    Task(description="frontend", priority=2),
    Task(description="database_migration", priority=0)  # Highest priority
]

# Start all in testing
for deploy in deployments:
    pipeline.add_item(deploy, progression_ids="testing")

print(f"Pipeline initialized: {pipeline}")
print(f"Testing stage: {len(pipeline.get_progression('testing'))} deployments")

# Promote high-priority to staging
testing = pipeline.get_progression("testing")
staging = pipeline.get_progression("staging")

for deploy_id in list(testing.order):  # Copy list to avoid modification during iteration
    deploy = pipeline.items[deploy_id]
    if deploy.priority == 0:  # High priority
        testing.remove(deploy_id)
        staging.append(deploy_id)
        print(f"Promoted: {deploy.description}")

# Final state
print(f"\nFinal state:")
for stage_name in ["testing", "staging", "production"]:
    stage = pipeline.get_progression(stage_name)
    items = [pipeline.items[i].description for i in stage.order]
    print(f"  {stage_name}: {items}")

## Summary

**Flow Dual-Pile Pattern**:
1. **Construction**: `Flow(items=..., name=...)`
2. **Progressions**: Named stages via `add_progression(Progression(name="stage"))`
3. **Items**: Shared pile via `add_item(item, progression_ids="stage")`
4. **M:N**: Items in multiple progressions (cross-cutting concerns)
5. **Named Access**: `flow.get_progression("name")` → O(1)
6. **Transitions**: Move UUIDs between progressions (state machine)
7. **Serialization**: Full round-trip with name index rebuild

**Design Rationale**:
- **Dual piles**: Separate lifecycle management (structure vs data)
- **UUID refs**: Serialization + lazy loading + distributed workflows
- **Named progressions**: Ergonomic + introspection + domain mapping
- **Independent ordering**: Same items, different views per progression

**Use Cases**: Task workflows, deployment pipelines, multi-stage processing, team views, priority lanes.